In [33]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import pandas as pd
import numpy as np
import json
from ssl import create_default_context
from datetime import datetime
from datetime import timedelta
import requests
#from getpass import getpass
import parametros

## Parametros de conexion para ES

In [34]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
)

## tomando fecha mas reciente del indice

Las alertas que traiga el Endpoint con fechas menores a la maxima en indice, se descartan.

In [35]:
total_docs = 0
try:
    response = es.search(
        index= parametros.cambium_a_h_index,
        body={"aggs" : {
                   "max_date": {"max": {"field": "time_raised", "format": "yyyy-MM-dd HH:mm:ss"}}
                }
             },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["aggregations"]
    fecha_max=response["aggregations"]["max_date"]['value_as_string']
except:
    fecha_max = '2010-01-01 00:00:01'
print("Fecha maxima en indice:",fecha_max)

Fecha maxima en indice: 2021-10-07 15:33:03


In [36]:
now = datetime.now()
new_date = now - timedelta(days=6)
format = "%Y%m%d"
format_ES = "%Y.%m.%d"
fecha_url = str(new_date.strftime(format))
fecha_hoy = str(now.strftime(format_ES))

In [37]:
aaa = str(now.strftime("%Y%m%d%H"))+'00'
fecha_ini = (datetime.strptime(aaa, '%Y%m%d%H%M')-timedelta(minutes=60)).strftime("%Y%m%d%H%M")
fecha_fin = (datetime.strptime(fecha_ini, '%Y%m%d%H%M')+timedelta(minutes=59)).strftime("%Y%m%d%H%M")

## Leyendo la APi cambium-alarm-history

In [38]:
datos_api = pd.DataFrame(columns=['ip','network','message','name','severity','source_type','status'
                                          ,'time_raised','time_cleared','site','duration','id','code','mac'
                                          ,'acknowledged_by','source','managed_account'])
for k in range(0,len(parametros.url_cambium)):
    token_aux = 'Bearer ' + parametros.cambium_token_aux[k]
    cabecera1 = { 'Content-Type': 'application/json', 'Authorization' : token_aux, 'accept' : '*/*' } 
    #url = parametros.url_cambium[k] + "alarms/history" + '?start_time='+ fecha_url
    url = parametros.url_cambium[k] + "alarms/history" + '?start_time=' + fecha_ini + '&stop_time=' + fecha_fin
    r = requests.get(url, headers = cabecera1, verify=False)
    if r.status_code == 200:
        res = json.loads(r.text)
        dato_param = res['paging']
        #se calcula los ciclos de la consulta paginada
        ciclos = int(round(dato_param['total']/100,0))
        i = 0
        while i < ciclos:
            i+=1
            offset = str(i)
            url2 = parametros.url_cambium[k] + "alarms/history" + '?offset=' + offset  + '&start_time=' + fecha_url 
            url = parametros.url_cambium[k] + "alarms/history" + '?start_time=' + fecha_ini + '&stop_time=' + fecha_fin
            r = requests.get(url2, headers = cabecera1, verify=False)
            if r.status_code == 200:
                #print("respuesta 200: ",r.text)
                res = json.loads(r.text)
                datos_api = datos_api.append(res['data'], ignore_index=True)
            else:
                #break
                print("Error de request:",r.status_code)

        pass
    else:
        print("el else saca esto: ",r.status_code)
        #exit()

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.252'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpoo

/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.224'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.224'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '100.123.26.224'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/urllib3/connectionpoo

In [39]:
datos_api

,ip,network,message,name,severity,source_type,status,time_raised,time_cleared,site,duration,id,code,mac,acknowledged_by,source,managed_account
0,172.25.31.12,SANTANDER,Device is offline.,Status,major,wifi-enterprise,cleared,2021-09-30T19:01:37-05:00,2021-10-01T08:40:34-05:00,46528-ZGYO409-VEREDA NOVILLERO CHI-SAN BENITO,49136,6156510ee5d4502d746c417c,STATUS,58:C1:7A:E9:C8:8A,,46528-ZGYO409-AP-EXT-1,
1,172.25.31.13,SANTANDER,Device is offline.,Status,major,wifi-enterprise,cleared,2021-09-30T19:01:37-05:00,2021-10-01T08:43:03-05:00,46528-ZGYO409-VEREDA NOVILLERO CHI-SAN BENITO,49285,6156514b76638c2d6e7403e7,STATUS,58:C1:7A:E9:C9:45,,46528-ZGYO409-AP-EXT-2,
2,172.28.117.37,META,Device is offline.,Status,major,wifi-enterprise,cleared,2021-09-30T19:02:53-05:00,2021-09-30T19:26:14-05:00,38765-ZGYO240 LA COOPERATIVA_FUENTE DE ORO,1400,61565159e5d4502d746c4444,STATUS,BC:E6:7C:4E:3E:36,,38765-ZGYO240-AP-EXT2,
3,172.28.117.35,META,Device is offline.,Status,major,wifi-enterprise,cleared,2021-09-30T19:02:53-05:00,2021-09-30T19:26:40-05:00,38765-ZGYO240 LA COOPERATIVA_FUENTE DE ORO,1426,61565159e5d4502d746c4445,STATUS,BC:E6:7C:EC:AA:E3,,38765-ZGYO240-AP-INT,
4,172.25.68.35,TOLIMA,Device is offline.,Status,major,wifi-enterprise,cleared,2021-09-30T19:02:53-05:00,2021-10-01T07:02:04-05:00,47804-ZZZY912-IBAGUE-PASTALES NUEVO,43150,6156515a76638c2d6e740456,STATUS,BC:E6:7C:ED:F3:14,,47804-ZZZY912-AP-INT,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,172.25.125.19,CESAR,Device is offline.,Status,major,wifi-enterprise,cleared,2021-10-01T10:10:24-05:00,2021-10-01T10:28:07-05:00,30376-ZZZY035 BECERRIL CGTO ESTADOS UNIDOS,1063,6157261062071231ea00be9e,STATUS,BC:E6:7C:EC:A9:F5,,30376-ZZZY035-AP-INT,
3196,172.25.97.245,ANTIOQUIA,Device is offline.,Status,major,wifi-enterprise,cleared,2021-10-01T10:10:25-05:00,2021-10-01T10:20:50-05:00,21818-ZZZY916 EL TAMACO TARAZA,625,6157261562071231ea00bef4,STATUS,BC:E6:7C:4F:47:27,,21818-ZZZY916-AP-EXT2,
3197,172.25.122.108,CORDOBA,Device is offline.,Status,major,wifi-enterprise,cleared,2021-10-01T10:11:24-05:00,2021-10-01T10:19:49-05:00,31291-ZGYO440 BUENAVISTA VEREDA LAS CRUCES,505,6157264e62071231ea00c1e9,STATUS,BC:E6:7C:4E:3E:95,,31291-ZGYO440-AP-EXT1,
3198,172.25.41.20,CORDOBA,Device is offline.,Status,major,wifi-enterprise,cleared,2021-10-01T10:13:53-05:00,2021-10-01T11:11:18-05:00,31418-ZGYO435 CAROLINA_CHIMA,3444,615726e0198b4c31f01eb05e,STATUS,BC:E6:7C:4E:3E:66,,31418-ZGYO435-AP-EXT1,


In [17]:
datos_api.to_excel("cambium_alarm_history_crudo.xlsx")

### Descartando datos de la API que ya están en el indice

In [40]:
try:
    datos_api['time_raised'] = (datos_api["time_raised"].str.split("T", n = 2, expand = True)[0])+' '+(datos_api["time_raised"].str.split("T", n = 2, expand = True)[1]).str.split("-", n = 1, expand = True)[0]
    datos_api['time_cleared'] = (datos_api["time_cleared"].str.split("T", n = 2, expand = True)[0])+' '+(datos_api["time_cleared"].str.split("T", n = 2, expand = True)[1]).str.split("-", n = 1, expand = True)[0]
except:
    print("Fecha:",now,"Error de request:",r.status_code)
    exit()

In [ ]:
datos_api.to_excel("cambium_alarm_history_crudo2.xlsx")

In [42]:
datos_api = datos_api.drop(datos_api[(datos_api['time_raised']<= fecha_max)].index)

In [31]:
datos_api.fillna('', inplace=True)

### Definiendo indice con fecha e insertando en ES

In [50]:
indice = parametros.cambium_a_h_index +'-'+ fecha_hoy

In [51]:
use_these_keys = ['ip','network','message','name','severity','source_type','status'
                  ,'time_raised','time_cleared','site','duration','id','code','mac'
                  ,'acknowledged_by','source','managed_account','@timestamp']
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

timestamp = datetime.now()
datos_api['@timestamp'] = timestamp.isoformat()
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_source": filterKeys(document),
            }
salida = helpers.bulk(es, doc_generator(datos_api))
print("Fecha:",now,"- Total documentos insertados:",salida[0])

Fecha: 2021-05-10 17:40:44.237766 - Total documentos insertados: 0
